<h1 style='text-align: center'>SQL Queries</h1>

## Getting Data From A SQL Database

### The Structure of a SQL Query

<img src='images/sql_statement.jpg'/>

#### SELECT

- What columns of data do we want? 
- Specify by column names or * (All) 

- Where are we getting our data from? (From statement)
- Specify by table name or use join statement to join multiple tables

##### JOIN 

- Use to join multiple tables 
- SELECT * FROM students join favorite_foods on students.id = favorite_foods.student_id
- SELECT * FROM students as s join favorite_foods as f on s.id = f.student_id

#### WHERE 
    - filters entire data set, as long as not using aggregate function

- apply some filter to the data you are querying 
- define criteria for data that should selected from the database 
- SELECT * from students WHERE age > 10 

#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city
- can apply function to that entire group i.e. MAX
- group by 1: group by first thing listed in SELECT statement 

#### HAVING

- Use to apply filter AFTER you have made your initial query 
- When should we use HAVING instead of WHERE

In [2]:
import sqlite3

conn = sqlite3.connect('tutorial.db')
c = conn.cursor()

In [3]:
# Load students.json file

#your code here
import json
f=open('students.json','r')
data=json.load(f)

In [4]:
data[0]

{'name': 'Sean Abu Wilson',
 'birthdate': '02/06',
 'siblings': 2,
 'Birthplace': 'Birmingham, AL',
 'yearsinnyc': 7.8,
 'favoritefood': 'guacamole'}

In [5]:
c.execute("DROP TABLE student_info")

In [6]:
droptable

NameError: name 'droptable' is not defined

In [7]:
# iterate over the data and insert each student into the DB


c.execute("""CREATE TABLE IF NOT EXISTS student_info 
        (name TEXT, birthdate TEXT, number_of_siblings INT, birth_place TEXT, years_in_nyc INT, favorite_food TEXT)""")


conn.commit()

In [8]:
# dynamic data entry example




def dynamic_data_entry(person):

    name = str(person['name'])
    birthdate = str(person['birthdate'])
    number_of_siblings = int(person['siblings'])
    birth_place = str(person['Birthplace'])
    years_in_nyc = float(person['yearsinnyc'])
    favorite_food = str(person['favoritefood'])

    c.execute('''INSERT INTO student_info (name, birthdate, number_of_siblings, birth_place, 
                 years_in_nyc, favorite_food) VALUES (?, ?, ?, ?, ?, ?)''',
                 (name, birthdate, number_of_siblings, birth_place, years_in_nyc, favorite_food))


              

    
   
# c.close
# conn.close()

In [9]:
for person in data:
    dynamic_data_entry(person)
conn.commit()


### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?
7. Which student was born closest to the cohort's graduation date?

In [10]:
c.execute("""SELECT name FROM student_info""").fetchall()

[('Sean Abu Wilson',),
 ('Tawab',),
 ('Bridget Boakye',),
 ('Cristina',),
 ('Danny (Haoyang) Yu',),
 ('Dustin_Breitner',),
 ('Gregory DeSantis',),
 ('Harrison Miller',),
 ('Jeremy Owens',),
 ('Jeremy Reikes',),
 ('Johnathan Peck',),
 ('Minna F',),
 ('Nabil Abbas',),
 ('Natalie Del Rossi',),
 ('Remy Canario',),
 ('Xristos Katsaros',)]

In [11]:
c.execute("""SELECT name, number_of_siblings FROM student_info WHERE number_of_siblings = (SELECT MAX(number_of_siblings) FROM student_info)""").fetchall()

[('Tawab', 4)]

In [21]:
c.execute("""SELECT name, COUNT(NAME) FROM student_info WHERE number_of_siblings == 0 
GROUP BY name""").fetchall()

[('Harrison Miller', 1),
 ('Jeremy Owens', 1),
 ('Natalie Del Rossi', 1),
 ('Remy Canario', 1)]

In [13]:
c.execute("""SELECT name, years_in_nyc from student_info ORDER BY years_in_nyc LIMIT 3""").fetchall()

[('Danny (Haoyang) Yu', 0), ('Harrison Miller', 0), ('Tawab', 0.5)]

In [31]:
c.execute("""SELECT COUNT(name)
          FROM student_info 
          WHERE birth_place LIKE '%york%'
          OR birth_place LIKE'%brooklyn%'
          OR birth_place LIKE '%island%'""").fetchall()

[(6,)]

In [38]:
c.execute("""UPDATE student_info SET favorite_food = 'Pizza' WHERE favorite_food LIKE '%izza';""")
c.execute("""SELECT favorite_food, COUNT(favorite_food) FROM student_info 
             GROUP BY favorite_food HAVING COUNT(favorite_food) > 1
             ORDER BY 1""").fetchall()

[('Pizza', 7)]

In [40]:
c.execute("""SELECT lower(favorite_food) favorite_food, COUNT(favorite_food) FROM student_info 
             GROUP BY favorite_food HAVING COUNT(favorite_food) > 1
             ORDER BY 1""").fetchall()

[('pizza', 7)]

In [16]:
c.execute("""SELECT name, birthdate FROM student_info WHERE birthdate LIKE '_9%'""").fetchall()

[('Bridget Boakye', '1990-07-13'),
 ('Danny (Haoyang) Yu', '1997-03-19'),
 ('Gregory DeSantis', '1987-11-23')]

In [17]:
c.execute("""SELECT name, birthdate FROM student_info""").fetchall()

[('Sean Abu Wilson', '02/06'),
 ('Tawab', '04/08/1998'),
 ('Bridget Boakye', '1990-07-13'),
 ('Cristina', '9/9/1990'),
 ('Danny (Haoyang) Yu', '1997-03-19'),
 ('Dustin_Breitner', '04/15/94'),
 ('Gregory DeSantis', '1987-11-23'),
 ('Harrison Miller', '10/2/85'),
 ('Jeremy Owens', '03/30/1990'),
 ('Jeremy Reikes', '1-/12/1995'),
 ('Johnathan Peck', '02-19-1994'),
 ('Minna F', '11-19-1996'),
 ('Nabil Abbas', '12-07-1994'),
 ('Natalie Del Rossi', '10-23-1994'),
 ('Remy Canario', '10-11-1988'),
 ('Xristos Katsaros', '03-24-1986')]

In [19]:
c.execute('''
SELECT name,
abs(julianday('2018-' || substr(birthdate,1,2) || '-' || substr(birthdate,4,2)) - julianday('2018-06-19'))
FROM student_info
ORDER BY 2
LIMIT 1
''').fetchall() 

[('Bridget Boakye', None)]